# Analysis Template: Inhibition
Updated 6/4/25 NF

In [1]:
#enables autoreloding of modules
%load_ext autoreload
%autoreload 2

from htbam_db_api.htbam_db_api import LocalHtbamDBAPI
from htbam_analysis.analysis.experiment import HTBAMExperiment

#enable inline plotting of matplotlib figures
%matplotlib inline

#set the figure format to SVG
%config InlineBackend.figure_format = 'svg'

## 1. Connect DB Api

In [2]:
### PARAMETERS:
EGFP_SLOPE = 91900.03
EGFP_SLOPE_CONC_UNITS = 'nM' #RFU/nM

root = './ic50_data/'
db_conn = LocalHtbamDBAPI(
    standard_curve_data_path= root + 'd3_2_StandardSeries_Analysis.csv',
    standard_name="standard_5-FAM", 
    standard_substrate="5-FAM", 
    standard_units="uM",
    kinetic_data_path= root+ 'd3_TitrationSeries_Analysis.csv',
    kinetic_name="kinetics_AVI4516", 
    kinetic_substrate="NSP 4/5", 
    kinetic_units="nM")

htbam_experiment = HTBAMExperiment(db_conn)


Connected to database.
Experiment found with the following runs:
['standard_5-FAM', 'kinetics_AVI4516', 'button_quant']


## 2. Enzyme Quant

In [3]:
from htbam_analysis.analysis.transform import transform_data

button_concentrations = transform_data(
    data_objs = [htbam_experiment.get_run('button_quant')],              # e.g. a Data2D or Data3D or Data4D instance
    expr=f'(a_luminance / {EGFP_SLOPE})',             # e.g. "(luminance - intercept) / slope"
    output_name='concentration'       # name of the new field, e.g. "concentration"
)

htbam_experiment.set_run('enzyme_concentrations', button_concentrations)     # save the fit results

Existing run data not found. Fetching from database.
Transforming 1 objects of type [<class 'htbam_db_api.data.Data2D'>] with shapes [(1792, 1)].


In [4]:
# from htbam_analysis.analysis.fitting import transform_data

# button_quant_data = htbam_experiment.get_run('button_quant')  # retrieve our button quant data
# button_concentrations = transform_data(button_quant_data, 
#                                        apply_to='luminance',
#                                        store_as='concentration',
#                                        function = lambda x: x / EGFP_SLOPE,  # convert RFU to nM
#                                        flatten = False,
#                                        data_type = 'concentration_data')                       # flatten array from (n_conc, n_time, n_chambers) to (n_chambers)
# htbam_experiment.set_run('enzyme_concentrations', button_concentrations)     # save the fit results

In [5]:
htbam_experiment.plot_enzyme_concentration_chip(analysis_name='enzyme_concentrations', units=EGFP_SLOPE_CONC_UNITS)

## 2. Product Standards

In [7]:
from htbam_analysis.analysis.fit import fit_luminance_vs_concentration

# For demonstration purposes, I'm doing this in 3 lines
# we can either keep it "exposed" like this (might be useful?)
# Or provide a simple wrapper fx that looks like:
# htbam_experiment.fit_standard_curve(experiment_name = 'standard_5-FAM', analysis_name='htbam_experiment')

standard_experiment_data = htbam_experiment.get_run('standard_5-FAM')       # retrieve our raw data
standard_fits = fit_luminance_vs_concentration(standard_experiment_data)    # perform a fit
htbam_experiment.set_run('standard_5-FAM_fits', standard_fits)              # save the fit results

Fit slopes for 1792 wells.
Elapsed 2.188 seconds.


In [8]:
htbam_experiment.plot_standard_curve_chip('standard_5-FAM_fits', 'standard_5-FAM')

## 3. Fit Initial Rates

In [9]:
# Calculate product concentrations from RFU data:
product_concentrations = transform_data(
    data_objs = [htbam_experiment.get_run('kinetics_AVI4516'), htbam_experiment.get_run('standard_5-FAM_fits')],
    expr=f'(a_luminance - b_intercept) / b_slope',             # e.g. "(luminance - intercept) / slope"
    output_name='concentration'       # name of the new field, e.g. "concentration"
)

htbam_experiment.set_run('kinetics_AVI4516_conc', product_concentrations)

Existing run data not found. Fetching from database.
Transforming 2 objects of type [<class 'htbam_db_api.data.Data4D'>, <class 'htbam_db_api.data.Data2D'>] with shapes [(12, 21, 1792, 1), (1792, 3)].


In [10]:
from htbam_analysis.analysis.fit import fit_concentration_vs_time

kinetics_concentrations = htbam_experiment.get_run('kinetics_AVI4516_conc')       # retrieve our raw data
kinetics_fits = fit_concentration_vs_time(kinetics_concentrations, start_timepoint = 1, end_timepoint=6)      # perform a fit, skipping the first timepoint
htbam_experiment.set_run('kinetics_AVI4516_conc_fits', kinetics_fits)                # save the fit results

Fit slopes for 1792 wells at 12 concentrations.
Elapsed 0.057 seconds.


In [18]:
htbam_experiment.plot_initial_rates_chip(analysis_name='kinetics_AVI4516_conc_fits', experiment_name='kinetics_AVI4516_conc')#, remove_0_point=True) # plot the initial rates

/home/freitas/workspace/htbam_sprint/htbam_analysis/src/htbam_analysis/analysis/experiment.py:228: RuntimeWarning:

Mean of empty slice



## 4. Filter initial rates

In [16]:
# These are the "Masks" we use to select which data we want.

from htbam_analysis.analysis.filter import filter_expression_cutoff, filter_initial_rates_positive_cutoff, filter_initial_rates_r2_cutoff, filter_standard_curve_r2_cutoff

kinetics_fits = htbam_experiment.get_run('kinetics_AVI4516_conc_fits')       # retrieve our raw data

# Initial Rates:
initial_rates_r2_mask =         filter_initial_rates_r2_cutoff(kinetics_fits, r2_cutoff=0.8) # 0.9 R2
initial_rates_positive_mask =   filter_initial_rates_positive_cutoff(kinetics_fits)          # positive slope

# Standard Curve:
standard_curve_r2_mask =        filter_standard_curve_r2_cutoff(standard_fits, kinetics_fits, r2_cutoff=0.9) # 0.9 R2

# Expression:
expression_mask =               filter_expression_cutoff(button_concentrations, kinetics_fits, expression_cutoff=1) # 1 nM

# Save the masks to the experiment
htbam_experiment.set_run('initial_rates_r2_mask', initial_rates_r2_mask)
htbam_experiment.set_run('initial_rates_positive_mask', initial_rates_positive_mask)
htbam_experiment.set_run('standard_curve_r2_mask', standard_curve_r2_mask)
htbam_experiment.set_run('expression_mask', expression_mask)



In [92]:
initial_rates_r2_mask.dep_var.shape

(12, 1792, 1)

In [116]:
# Plot the masks (Do one at a time)
#htbam_experiment.plot_mask_chip(mask_name='initial_rates_r2_mask')
htbam_experiment.plot_mask_chip(mask_name='initial_rates_positive_mask')
#htbam_experiment.plot_mask_chip(mask_name='standard_curve_r2_mask')
#htbam_experiment.plot_mask_chip(mask_name='expression_mask')

In [30]:
# Apply the masks to the fits and save the results
htbam_experiment.apply_mask(run_name='kinetics_AVI4516_conc_fits', 
                            dep_variables = ['slope', 'intercept'], 
                            save_as = 'kinetics_AVI4516_conc_fits_masked',
                            mask_names = ['initial_rates_r2_mask', 'initial_rates_positive_mask', 'standard_curve_r2_mask', 'expression_mask'])


In [31]:
htbam_experiment.plot_initial_rates_chip(analysis_name='kinetics_AVI4516_conc_fits_masked', experiment_name='kinetics_AVI4516_conc')#, remove_0_point=True) # plot the initial rates

/home/freitas/workspace/htbam_sprint/htbam_analysis/src/htbam_analysis/analysis/experiment.py:229: RuntimeWarning:

Mean of empty slice



In [76]:
htbam_experiment.plot_initial_rates_vs_concentration_chip(analysis_name='kinetics_AVI4516_conc_fits_masked', x_log=True) # plot the initial rates

/home/freitas/workspace/htbam_sprint/htbam_analysis/src/htbam_analysis/analysis/experiment.py:286: RuntimeWarning:

Mean of empty slice



## 5. Fit Inhibition Constant:

In [57]:
# I've written MM here, but it will be IC50
from htbam_analysis.analysis.fit import fit_initial_rates_vs_concentration_with_function, mm_model, inhibition_model

kinetics_fits_masked = htbam_experiment.get_run('kinetics_AVI4516_conc_fits_masked')       # retrieve our raw data
IC50_fits, IC50_pred_data = fit_initial_rates_vs_concentration_with_function(data = kinetics_fits_masked,
                                model_func = inhibition_model)
htbam_experiment.set_run('kinetics_AVI4516_IC50_fits', IC50_fits)                # save the fit results
htbam_experiment.set_run('kinetics_AVI4516_IC50_pred_data', IC50_pred_data)    # save the predicted data

Successfully fit nonlinear model for 973 wells.
Elapsed 1.005 seconds.


In [117]:
from htbam_analysis.analysis.filter import filter_r2_cutoff
# Filter out crappy R2 values: 
IC50_fits = htbam_experiment.get_run('kinetics_AVI4516_IC50_fits')
IC50_fits_mask = filter_r2_cutoff(IC50_fits, r2_cutoff=0.75)  # 0.8 R2

htbam_experiment.set_run('IC50_R2_mask', IC50_fits_mask)  # save the fit results

htbam_experiment.plot_mask_chip(mask_name='IC50_R2_mask')

[2025-06-06 17:36:06,853] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/freitas/miniforge3/envs/htbam_sprint/lib/python3.9/site-packages/dash/dash.py", line 1301, in dispatch
    cb = self.callback_map[output]
KeyError: '..graph-tooltip.show...graph-tooltip.bbox...graph-tooltip.children..'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/freitas/miniforge3/envs/htbam_sprint/lib/python3.9/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/freitas/miniforge3/envs/htbam_sprint/lib/python3.9/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/freitas/miniforge3/envs/htbam_sprint/lib/python3.9/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/freitas/miniforge3/envs/htbam_sprint/

In [104]:
# Apply the mask to the fits and save the results:
htbam_experiment.apply_mask(run_name='kinetics_AVI4516_IC50_fits', 
                            dep_variables = ['r_min', 'r_max', 'ic50'], 
                            save_as = 'kinetics_AVI4516_IC50_fits_masked',
                            mask_names = ['IC50_R2_mask'])

In [129]:
htbam_experiment.plot_ic50_chip(analysis_name='kinetics_AVI4516_conc_fits_masked', 
                                model_fit_name='kinetics_AVI4516_IC50_fits_masked',
                                model_pred_data_name='kinetics_AVI4516_IC50_pred_data',
                                x_log=True) # plot the initial rates

/home/freitas/workspace/htbam_sprint/htbam_analysis/src/htbam_analysis/analysis/experiment.py:366: RuntimeWarning:

All-NaN slice encountered

/home/freitas/workspace/htbam_sprint/htbam_analysis/src/htbam_analysis/analysis/experiment.py:367: RuntimeWarning:

All-NaN slice encountered



In [ ]:
htbam_experiment.export_run_data_raw(run_name='kinetics_AVI4516_IC50_fits_masked')
htbam_experiment.export_run_data_processed(run_name='kinetics_AVI4516_IC50_fits_masked')

Run data exported to /home/freitas/workspace/htbam_sprint/htbam_db_api/examples/kinetics_AVI4516_IC50_fits_masked_data.csv


AssertionError: run_data must be of type Data2D.

In [ ]:
elapsed = time.time() - start
print(f"Elapsed time: {elapsed:.2f} seconds")

In [ ]:
# TOD

In [ ]:
#htbam_experiment.plot_MM_chip('kinetics_AVI4516_MM', show_average_fit=True)

## 6. Export to CSV

In [ ]:
#htbam_experiment.export_ic50_result_csv(path_to_save=root,
#                                        run_name='kinetics_0')